<a href="https://colab.research.google.com/github/Dat-291020/BTLAI/blob/main/DA_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#/content/drive/MyDrive/DATN/Source Code/data_4/data.csv

In [ ]:
#read.py
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

def load_csv(data_dir, features = []):
    df = pd.read_csv(data_dir)
    if len(features) > 0:
        df = df[features]
    return df

def split(df, features, scaler, valid_date, test_date, end_date, time_lag=1, freq='D'):
    X = df[features]
    X[features] = scaler.transform(X)
    train = X.copy()[X.index < valid_date]
    train = window_generate(train, features, time_lag=time_lag, freq=freq)
    valid = X.copy()[(X.index < test_date) & (X.index >= valid_date)]
    valid = window_generate(valid, features, time_lag=time_lag, freq=freq)
    test = X.copy()[(X.index < end_date) & (X.index >= test_date)]
    test = window_generate(test, features, time_lag=time_lag, freq=freq)
    return train, valid, test

def split_y(df, targets, scaler, valid_date, test_date, end_date, horizon=1, freq='D'):
    X = df[targets]
    X[targets] = scaler.transform(X)
    m = targets
    for i in range(len(targets)):
        m[i]=targets[i]+"+"+str(horizon)
    X=X.set_axis(m, axis='columns')
    train = X.copy()[X.index < valid_date]
    train = train.shift(periods=-horizon, freq=freq)
    valid = X.copy()[(X.index < test_date) & (X.index >= valid_date)]
    valid = valid.shift(periods=-horizon, freq=freq)
    test = X.copy()[(X.index < end_date) & (X.index >= test_date)]
    test = test.shift(periods=-horizon, freq=freq)
    return train, valid, test

def window_generate(df, features, time_lag, freq='D'):
    data = df.copy()[features]
    result = pd.DataFrame()
    for i in range(-time_lag+1, 1):
        columns = []
        data_i = data.shift(periods=-i, freq=freq)
        for j in range(len(features)):
            columnsj = data.columns[j] + '_%d'%(i)
            columns.append(columnsj)
        data_i = data_i.set_axis(columns, axis=1, inplace=False)
        result = pd.concat([result, data_i], axis = 1)
    return result

def power_set(seq):
    if len(seq)<=1:
        yield seq
        yield []
    else:
        for item in power_set(seq[1:]):
            yield [seq[0]]+item
            yield item

In [ ]:
#model.py
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Input, LSTM, Dense, GRU, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Flatten, Activation, Reshape, Concatenate
from tensorflow.keras.models import Model

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

def normal_lstm(features_count = 1, lag_time = 1):
    x = Input(shape = (lag_time, features_count))
    lstm = LSTM(32)(x)
    dense1 = Dense(16)(lstm)
    output = Dense(1)(dense1)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mape', 'mae'])
    model.summary()
    return model

def stack_lstm(features_count = 1, lag_time = 1):
    x = Input(shape = (lag_time, features_count))
    lstm1 = LSTM(64, return_sequences = True)(x)
    lstm2 = LSTM(32)(lstm1)
    dense1 = Dense(64, name = 'spec_out0')(lstm2)
    output = Dense(1)(dense1)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mape', 'mae'])
    model.summary()
    return model

def mtl_mtv_lstm(features_count, addition_features, lag_time, batch_size=32):
    x = Input(shape = (lag_time, features_count))
    lstm1 = LSTM(16, return_sequences = True)(x)
    lstm2 = LSTM(32, return_sequences = True)(lstm1)
    shared_dense = Dense(64)(lstm2)
    # main task
    sub1 = GRU(units=(lag_time*addition_features), name="task1")(shared_dense)
    #addition task for main imf
    sub2 = LSTM(units=16, name="task2")(shared_dense)
    sub3 = LSTM(units=16, name="task3")(shared_dense)
    # merge sub1 with addition input imfs
    sub1 = Reshape((lag_time, addition_features))(sub1)
    addition_input = Input(shape=(lag_time, addition_features), name='add_input')
    concate = Concatenate(axis=-1)([sub1, addition_input])
    #perform mtl
    out1 = Dense(8, name="spec_out1")(concate)
    out1 = Flatten()(out1)
    out1 = Dense(1, name="out1")(out1)

    out2 = Dense(8, name="spec_out2")(sub2)
    out2 = Dense(1, name="out2")(out2)

    out3 = Dense(1, name="spec_out3")(sub3)
    out3 = Dense(1, name="out3")(out3)

    outputs = [out1, out2, out3]
    # define model
    model = Model(inputs = [x, addition_input], outputs = outputs)
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mape', 'mae'], loss_weights=[0.5, 0.01, 0.01])
    model.summary()
    return model

def model_mtl_mtv(horizon=1, nb_train_samples=512, batch_size=32, 
                  feature_count=11, lag_time=6, auxiliary_feature_count=12):

    x = Input(shape=(lag_time, feature_count), name="input_layer")

    lstm1 = GRU(16, return_sequences=True)(x)
    # lstm2 = GRU(32, return_sequences=True)(mp)
    lstm2 = GRU(32, return_sequences=True)(lstm1)

    shared_dense = Dense(64, name="shared_layer")(lstm2)

    ## sub1 is main task; units = reshape dimension multiplication
    sub1 = GRU(units=(lag_time*auxiliary_feature_count), name="task1")(shared_dense)
    sub2 = GRU(units=16, name="task2")(shared_dense)
    sub3 = GRU(units=16, name="task3")(shared_dense)

    sub1 = Reshape((lag_time, auxiliary_feature_count))(sub1)
    auxiliary_input = Input(shape=(lag_time, auxiliary_feature_count), name='aux_input')

    concate = Concatenate(axis=-1)([sub1, auxiliary_input])
    # out1_gp = Dense(1, name="out1_gp")(sub1)
    out1 = Dense(8, name="spec_out1")(concate)
    out1 = Flatten()(out1)
    out1 = Dense(1, name="out1")(out1)

    out2 = Dense(8, name="spec_out2")(sub2)
    out2 = Dense(1, name="out2")(out2)

    out3 = Dense(1, name="spec_out3")(sub3)
    out3 = Dense(1, name="out3")(out3)

    outputs = [out1, out2, out3]

    model = Model(inputs=[x, auxiliary_input], outputs=outputs)

    # adam optimizsor is good
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mape', 'mae'], loss_weights=[0.5, 0.01, 0.01])
    model.summary()
    return model

def mtl_mtv_GRU(horizon=1, nb_train_samples=512, batch_size=32, targets=[], 
                  feature_count=11, lag_time=6, auxiliary_feature_count=12):
    x = Input(shape=(lag_time, feature_count), name="input_layer")

    lstm1 = GRU(16, return_sequences=True)(x)
    # lstm2 = GRU(32, return_sequences=True)(mp)
    lstm2 = GRU(32, return_sequences=True)(lstm1)

    shared_dense = Dense(64, name="shared_layer")(lstm2)

    ## sub1 is main task; units = reshape dimension multiplication
    sub = []
    for i in range(len(targets)):
        if i==0:
            subi = GRU(units=(lag_time*auxiliary_feature_count), name="task0")(shared_dense)
        else:
            subi = GRU(units=16, name="task"+str(i))(shared_dense)
        sub.append(subi)

    sub[0] = Reshape((lag_time, auxiliary_feature_count))(sub[0])
    auxiliary_input = Input(shape=(lag_time, auxiliary_feature_count), name='aux_input')

    concate = Concatenate(axis=-1)([sub[0], auxiliary_input])
    # out1_gp = Dense(1, name="out1_gp")(sub1)
    out1 = Dense(8, name="spec_out0")(concate)
    out1 = Flatten()(out1)
    out1 = Dense(1, name="out0")(out1)
    
    out=[]
    out.append(out1)
    for i in range(1, len(targets)):
        outi = Dense(8, name="spec_out"+str(i))(sub[i])
        outi = Dense(1, name="out"+str(i))(outi)
        out.append(outi)

    outputs = out

    model = Model(inputs=[x, auxiliary_input], outputs=outputs)
    
    weights=[]
    for i in range(len(out)):
        if i==0:
            weights.append(0.5)
        else:
            weights.append(0.1)

    # adam optimizsor is good
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mape', 'mae'], loss_weights=weights)
    # model.summary()
    return model

abc


In [ ]:
#from preprocessing.read import load_csv, window_generate, split, split_y, power_set
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoLars
#from model.model import normal_lstm, stack_lstm, mtl_mtv_lstm, model_mtl_mtv, mtl_mtv_GRU
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score, mean_absolute_error, explained_variance_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from keras.models import Model

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit, TimeSeriesSplit


In [ ]:
# load data and set the datetime index for data frame
datadir = '/content/drive/MyDrive/DATN/Source Code/data_4/data.csv'
features = []
data = load_csv(datadir, features = features)
data = data.set_index(pd.to_datetime(data['date'], format='%m/%d/%Y'))
data = data.drop(['date'], axis=1)
data = data.dropna()


In [ ]:
data.head()

,x1,x2,x3,y
date,,,,
1994-01-01,2007,897,2118,4653.47
1994-01-02,2534,2161,1752,24191.18
1994-01-03,2085,1728,1975,19187.24
1994-01-04,502,1184,2445,19454.29
1994-01-05,1233,581,1087,14488.55


In [ ]:

# preparing the train, valid and test data
## set the milestone for dividing data by time
## đặt các mốc thời gian để chia dữ liệu thành train/validatate/test
valid_date = "1999-01-01 00:00:00"
test_date = "2001-01-01 00:00:00"
end_date = "2003-01-01 00:00:00"
## set the lag time and horizon
LAG = 3
HORIZON = 1
## data scaler (công cụ chuẩn hóa dữ liệu)
X_scaler = StandardScaler()
y_scaler = StandardScaler()
main_target_scaler = StandardScaler()

result = pd.DataFrame()
result[['algorithm', 'LAG', 'Horizon', 'rmse', 'mae', 'r2', 'nse']] =["","","","","","",""]

In [ ]:
# =========================================================

X_scaler = StandardScaler()
y_scaler = StandardScaler()
main_target_scaler = StandardScaler()

input_features = data.columns[:-1] # tất cả các cột trừ cột y
targets = [data.columns[-1]] # cột y

X = data[input_features]
y = data[targets]

## tạo train data
X_train = X.copy().loc[X.index < valid_date]
y_train = y.copy().loc[y.index < valid_date]

# fit công cụ chuẩn hóa theo dữ liệu train
X_scaler.fit(X_train)
y_scaler.fit(y_train)
main_target_scaler.fit(y_train[targets[0]].values.reshape(-1,1))

# dùng hàm tự định nghĩa split (trong utils/read.py) để chia dữ liệu thành
# train/valid/test data, đồng thời tạo các chuỗi dữ liệu độ dài LAG để phục vụ dự báo
X_tr, X_v, X_t = split(data, input_features, X_scaler, 
                                 valid_date, test_date, end_date, time_lag=LAG, freq='D')
y_tr, y_v, y_t = split_y(data, targets, y_scaler, 
                                 valid_date, test_date, end_date, horizon=HORIZON, freq='D')

# ==========================================================

In [ ]:
# vì lstm cần cả train, valid và test nên thực hiện như thế này
train = pd.concat([X_tr, y_tr], axis=1) # gộp chung dữ liệu đầu vào và dãy đầu ra
train = train.loc[:,~train.columns.duplicated()].copy()#loại dữ liệu lặp
train = train.dropna()#loại các dòng có ô trống

valid = pd.concat([X_v, y_v], axis=1)
valid = valid.loc[:,~valid.columns.duplicated()].copy()
valid = valid.dropna()

test = pd.concat([X_t, y_t], axis=1)
test = test.loc[:,~test.columns.duplicated()].copy()
test = test.dropna()

in_features = X_tr.columns
out_features = y_tr.columns

# train, valid, test for lstm
Xtrain = train[in_features].values
Xtrain = Xtrain.reshape(Xtrain.shape[0], LAG, len(input_features))
ytrain = []
for feature in out_features:
    ytrain.append(train[feature].values.reshape(-1,1))

Xvalid = valid[in_features].values
Xvalid = Xvalid.reshape(Xvalid.shape[0], LAG, len(input_features))
yvalid = []
for feature in out_features:
    yvalid.append(valid[feature].values.reshape(-1,1))

Xtest = test[in_features].values
Xtest = Xtest.reshape(Xtest.shape[0], LAG, len(input_features))
ytest = []
for feature in out_features:
    ytest.append(test[feature].values.reshape(-1,1))

# Train, Test for Lassor, RF, SVR...
## làm riêng vì với các thuật toán này không cần validate data
XTrain = np.concatenate([Xtrain.reshape(-1, len(in_features)), 
                          Xvalid.reshape(-1, len(in_features))], axis=0)
XTest = Xtest.reshape(-1, len(in_features))
yTrain = np.concatenate([ytrain[0], yvalid[0]], axis=0).ravel()
yTest = ytest[0].ravel()

# ==========================================================

# nash-sutcliffe efficiency
def nse(targets,predictions):
    return 1-(np.sum((targets-predictions)**2)/np.sum((targets-np.mean(predictions))**2))

# ==========================================================

## gọi công cụ chia dữ liệu để chia 5 fold với mỗi fold có cỡ test = 365
cv = TimeSeriesSplit(n_splits=5, test_size=300)

In [ ]:
# LASSO
## tạo vùng cho các tham số để lặp và chọn tham số tốt nhất
alpha_range = np.linspace(0, 0.3, 31)
lasso_param_grid = dict(alpha = alpha_range)
## gọi hàm chọn tham số (GridSearchCV)
grid_lasso = GridSearchCV(LassoLars(), param_grid=lasso_param_grid, cv=cv,
                          scoring = 'neg_mean_squared_error',
                          verbose = 0)
## fit hàm chọn tham số với dữ liệu
grid_lasso.fit(XTrain, yTrain)
## trả kết quả
print("The best parameters of LassorLars are %s with a score of %0.2f"
    % (grid_lasso.best_params_, grid_lasso.best_score_))
## gọi mô hình dự báo
model_lasso = LassoLars(alpha=grid_lasso.best_params_['alpha'], normalize=False)
## dự báo
yPredLasso = model_lasso.fit(XTrain, yTrain).predict(XTest)
## đảo ngược chuẩn hóa để đưa mục tiêu dự báo về giá trị ban đầu
y_test_lasso = main_target_scaler.inverse_transform(yTest.reshape(-1,1))
y_pred_lasso = main_target_scaler.inverse_transform(yPredLasso.reshape(-1,1))
## các yếu tố đánh giá
print('lasso')
print('rmse: ' + str(mean_squared_error(y_test_lasso, y_pred_lasso, squared=False)))
print('r2: '+ str(r2_score(y_test_lasso, y_pred_lasso)))
print('mae: ' + str(mean_absolute_error(y_test_lasso, y_pred_lasso)))
print('mape: '+ str(mean_absolute_percentage_error(y_test_lasso, y_pred_lasso)))
print('nse: ' + str(nse(y_test_lasso, y_pred_lasso)))
## thêm kq vào dataframe result
result.loc[result.shape[0]] = ['LASSO', LAG, HORIZON,
                               mean_squared_error(y_test_lasso, y_pred_lasso, squared=False),
                               mean_absolute_error(y_test_lasso, y_pred_lasso),
                               r2_score(y_test_lasso, y_pred_lasso),
                               nse(y_test_lasso, y_pred_lasso)]


In [ ]:
# svr
C_range = np.logspace(-3, 3, 7)
gamma_range = np.logspace(-5, 2, 8)
param_grid = dict(gamma=gamma_range, C=C_range)
grid = GridSearchCV(SVR(), param_grid=param_grid, cv=cv,
                    scoring = 'neg_mean_absolute_percentage_error',
                    verbose=0)
grid.fit(XTrain, yTrain)

print("The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_))

model_svr = SVR(kernel='rbf', C=grid.best_params_['C'], 
          gamma = grid.best_params_['gamma'])

yPredSVR = model_svr.fit(XTrain, yTrain).predict(XTest)
y_test_svr = main_target_scaler.inverse_transform(yTest.reshape(-1,1))
y_pred_svr = main_target_scaler.inverse_transform(yPredSVR.reshape(-1,1))

print('svr')
print('rmse: ' + str(mean_squared_error(y_test_svr, y_pred_svr, squared=False)))
print('r2: '+ str(r2_score(y_test_svr, y_pred_svr)))
print('mae: ' + str(mean_absolute_error(y_test_svr, y_pred_svr)))
print('mape: '+ str(mean_absolute_percentage_error(y_test_svr, y_pred_svr)))
print('nse: ' + str(nse(y_test_svr, y_pred_svr)))

result.loc[result.shape[0]] = ['SVR', LAG, HORIZON,
                               mean_squared_error(y_test_svr, y_pred_svr, squared=False),
                               mean_absolute_error(y_test_svr, y_pred_svr),
                               r2_score(y_test_svr, y_pred_svr),
                               nse(y_test_svr, y_pred_svr)]


The best parameters are {'C': 0.001, 'gamma': 0.1} with a score of -1.09
svr
rmse: 3446.8307620108403
r2: -0.1644553390955863
mae: 2781.2825852332303
mape: 0.12141039838292514
nse: 0.11222913780452226


In [ ]:
# rf
n_estimators_range = [100, 200, 500, 1000]
max_depth_range = np.array(range(5, 25, 5))
rf_param_grid = dict(n_estimators=n_estimators_range, max_depth=max_depth_range)

grid_rf = GridSearchCV(RandomForestRegressor(), param_grid=rf_param_grid, cv=cv,
                    scoring = 'neg_mean_squared_error',  verbose=0)
grid_rf.fit(XTrain, yTrain)
print("The best parameters of randomforest are %s with a score of %0.2f"
    % (grid_rf.best_params_, grid_rf.best_score_))

model_rf = RandomForestRegressor(n_estimators=grid_rf.best_params_['n_estimators'],
                                 max_depth=grid_rf.best_params_['max_depth'])
y_pred_rf = model_rf.fit(XTrain, yTrain).predict(XTest)
y_test_rf = main_target_scaler.inverse_transform(yTest.reshape(-1,1))
y_pred_rf = main_target_scaler.inverse_transform(y_pred_rf.reshape(-1,1))

print('rf')
print('rmse: ' + str(mean_squared_error(y_test_rf, y_pred_rf, squared=False)))
print('r2: '+ str(r2_score(y_test_rf, y_pred_rf)))
print('mae: ' + str(mean_absolute_error(y_test_rf, y_pred_rf)))
print('mape: '+ str(mean_absolute_percentage_error(y_test_rf, y_pred_rf)))
print('nse: ' + str(nse(y_test_rf, y_pred_rf)))

result.loc[result.shape[0]] = ['RF', LAG, HORIZON,
                               mean_squared_error(y_test_rf, y_pred_rf, squared=False),
                               mean_absolute_error(y_test_rf, y_pred_rf),
                               r2_score(y_test_rf, y_pred_rf),
                               nse(y_test_rf, y_pred_rf)]


The best parameters of randomforest are {'max_depth': 20, 'n_estimators': 1000} with a score of -0.15
rf
rmse: 1953.3097392390073
r2: 0.6260400035929276
mae: 1850.6891976224567
mape: 0.08487467870720829
nse: 0.7200267206918238


In [ ]:

#lstm
## gọi các công cụ hỗ trợ
earlystop = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
file_path = 'checkpoint/weights-improvement-{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(monitor="val_loss",
                              filepath=file_path,
                              verbose=0,
                              save_weight_only=True,
                              save_best_only=True)
## gọi hàm tự định nghĩa để khai báo mô hình lstm (trong model/model.py)
model_lstm = stack_lstm(features_count=len(input_features), lag_time=LAG)
## train
model_lstm.fit(Xtrain, ytrain[0], 
          batch_size=32, 
          epochs=500, 
          validation_data=(Xvalid, yvalid[0]),
          callbacks=[earlystop, checkpoint],
          verbose = 2)
## dự báo
y_pred_lstm = model_lstm.predict(Xtest)
y_test_lstm = main_target_scaler.inverse_transform(ytest[0].reshape(-1,1))
y_pred_lstm = main_target_scaler.inverse_transform(y_pred_lstm.reshape(-1,1))

print('lstm')
print('rmse: ' + str(mean_squared_error(y_test_lstm, y_pred_lstm, squared=False)))
print('r2: '+ str(r2_score(y_test_lstm, y_pred_lstm)))
print('mae: ' + str(mean_absolute_error(y_test_lstm, y_pred_lstm)))
print('mape: '+ str(mean_absolute_percentage_error(y_test_lstm, y_pred_lstm)))
print('nse: ' + str(nse(y_test_lstm, y_pred_lstm)))

result.loc[result.shape[0]] = ['lstm', LAG, HORIZON,
                               mean_squared_error(y_test_lstm, y_pred_lstm, squared=False),
                               mean_absolute_error(y_test_lstm, y_pred_lstm),
                               r2_score(y_test_lstm, y_pred_lstm),
                               nse(y_test_lstm, y_pred_lstm)]

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 3)]            0         
                                                                 
 lstm (LSTM)                 (None, 3, 64)             17408     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 spec_out0 (Dense)           (None, 64)                2112      
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 32,001
Trainable params: 32,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
57/57 - 6s - loss: 0.8649 - mse: 0.8649 - map